In [ ]:
import zipfile
import os
import json
import shutil
from sklearn.model_selection import train_test_split
from ultralytics import YOLO
import torch
import cv2
import numpy as np
import torchvision
from torchvision.transforms import functional as F

In [ ]:
import zipfile
import os

# Step 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Define the base paths for your project directories
zip_files_path = '/content/drive/MyDrive/DL_PROJECT'  # Path to your zip files in Google Drive
output_base_dir = '/content/extracted_images'  # Base directory for extracted images and other files
yolo_labels_dir = os.path.join(output_base_dir, 'yolo/labels')  # Directory to store YOLO labels
dataset_split_dir = os.path.join(output_base_dir, 'dataset_split')  # Directory for dataset split (train, val, test)

# Create directories if they don't exist
os.makedirs(yolo_labels_dir, exist_ok=True)
os.makedirs(os.path.join(dataset_split_dir, 'train/images'), exist_ok=True)
os.makedirs(os.path.join(dataset_split_dir, 'train/labels'), exist_ok=True)
os.makedirs(os.path.join(dataset_split_dir, 'val/images'), exist_ok=True)
os.makedirs(os.path.join(dataset_split_dir, 'val/labels'), exist_ok=True)
os.makedirs(os.path.join(dataset_split_dir, 'test/images'), exist_ok=True)
os.makedirs(os.path.join(dataset_split_dir, 'test/labels'), exist_ok=True)

print("Directory structure created.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Directory structure created.


In [ ]:
# Step 2: Extract all zip files from Google Drive to 'extracted_images' directory in Colab
zip_files = [
    os.path.join(zip_files_path, 'fish_tray_images_2021_04_01.zip'),
    os.path.join(zip_files_path, 'fish_tray_images_2021_04_20.zip'),
    os.path.join(zip_files_path, 'fish_tray_images_2021_05_01.zip'),
    os.path.join(zip_files_path, 'fish_tray_images_2021_05_17.zip'),
    os.path.join(zip_files_path, 'fish_tray_images_2021_06-09.zip')
]

# Extract each zip file
for file in zip_files:
    with zipfile.ZipFile(file, 'r') as zip_ref:
        zip_ref.extractall(output_base_dir)

print("All files extracted to 'extracted_images' directory.")

All files extracted to 'extracted_images' directory.


In [ ]:
# Verify that each image has a corresponding YOLO label file
image_files = [f for f in os.listdir(output_base_dir) if f.endswith('.jpg')]
image_files_with_labels = []

for img in image_files:
    label_path = os.path.join(yolo_labels_dir, img.replace('.jpg', '.txt'))
    if os.path.exists(label_path) and os.path.getsize(label_path) > 0:
        image_files_with_labels.append(img)
    else:
        print(f"Skipping {img} as it has no valid annotation.")

print(f"Total images with annotations: {len(image_files_with_labels)}")

Skipping 20_04_21-B16.jpg as it has no valid annotation.
Skipping 10_05_21-B4.jpg as it has no valid annotation.
Skipping 20_04_21-B10.jpg as it has no valid annotation.
Skipping 16_04_21-B.37.jpg as it has no valid annotation.
Skipping 17_05_21-B5.jpg as it has no valid annotation.
Skipping 08_04_21-B.22.jpg as it has no valid annotation.
Skipping 13_04_21-B.13.jpg as it has no valid annotation.
Skipping 12_04_21-B.14.jpg as it has no valid annotation.
Skipping 10_05_21-B15.jpg as it has no valid annotation.
Skipping 19_05_21-B10.jpg as it has no valid annotation.
Skipping 19_04_21-B21.jpg as it has no valid annotation.
Skipping 22_04_21-B4.jpg as it has no valid annotation.
Skipping 13_04_21-B.35.jpg as it has no valid annotation.
Skipping 13_04_21-B.12.jpg as it has no valid annotation.
Skipping 4_05_21-B36.jpg as it has no valid annotation.
Skipping 20_05_21-B16.jpg as it has no valid annotation.
Skipping 24_05_21-B20.jpg as it has no valid annotation.
Skipping 23_04_21-B14.jpg as 

In [ ]:
# Function to convert COCO bounding boxes to YOLO format
def coco_to_yolo(coco_bbox, img_width, img_height):
    x_min, y_min, box_width, box_height = coco_bbox
    x_center = (x_min + box_width / 2) / img_width
    y_center = (y_min + box_height / 2) / img_height
    box_width /= img_width
    box_height /= img_height
    return [x_center, y_center, box_width, box_height]

In [ ]:
# Step 3: Load COCO Annotations and Convert to YOLO Format
coco_annotation_file = os.path.join(zip_files_path, 'coco_format_fish_data.json')
with open(coco_annotation_file, 'r') as f:
    coco_data = json.load(f)

images = coco_data['images']
annotations = coco_data['annotations']
categories = coco_data['categories']
image_dict = {img['id']: {'file_name': img['file_name'], 'width': img['width'], 'height': img['height']} for img in images}

# Convert COCO annotations to YOLO format
for ann in annotations:
    img_id = ann['image_id']
    image_info = image_dict[img_id]
    img_width, img_height = image_info['width'], image_info['height']
    yolo_bbox = coco_to_yolo(ann['bbox'], img_width, img_height)
    class_id = ann['category_id']
    label_file_name = image_info['file_name'].replace('.jpg', '.txt')
    label_file_path = os.path.join(yolo_labels_dir, label_file_name)
    with open(label_file_path, 'a') as label_file:
        label_file.write(f"{class_id} {' '.join(map(str, yolo_bbox))}\n")

print("COCO to YOLO conversion completed.")

COCO to YOLO conversion completed.


In [ ]:
# Step 4: Split Dataset into Train, Validation, and Test Sets
image_files = [f for f in os.listdir(output_base_dir) if f.endswith('.jpg')]
image_files = [f for f in image_files if os.path.exists(os.path.join(yolo_labels_dir, f.replace('.jpg', '.txt')))]
train_images, test_images = train_test_split(image_files_with_labels, test_size=0.2, random_state=42)
train_images, val_images = train_test_split(train_images, test_size=0.1, random_state=42)

# Function to copy images and labels to train/val/test directories
def copy_files(image_list, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    label_output_dir = output_dir.replace('images', 'labels')
    os.makedirs(label_output_dir, exist_ok=True)
    for image_file in image_list:
        shutil.copy(os.path.join(output_base_dir, image_file), os.path.join(output_dir, image_file))
        label_file = image_file.replace('.jpg', '.txt')
        shutil.copy(os.path.join(yolo_labels_dir, label_file), os.path.join(label_output_dir, label_file))

# Copy files into train, val, test directories
copy_files(train_images, os.path.join(dataset_split_dir, 'train/images'))
copy_files(val_images, os.path.join(dataset_split_dir, 'val/images'))
copy_files(test_images, os.path.join(dataset_split_dir, 'test/images'))
print("Dataset splitting complete.")

Dataset splitting complete.


In [ ]:
# Paths to your image directories
val_images_dir = '/content/extracted_images/dataset_split/val/images'
train_images_dir = '/content/extracted_images/dataset_split/train/images'
test_images_dir = '/content/extracted_images/dataset_split/test/images'

def check_and_fix_images(image_dir):
    for img_file in os.listdir(image_dir):
        img_path = os.path.join(image_dir, img_file)
        try:
            # Attempt to load the image
            img = cv2.imread(img_path)
            # If loading failed, img will be None
            if img is None:
                print(f"Corrupted image found: {img_file}. Attempting to delete.")
                os.remove(img_path)
            else:
                # Re-save the image to "clean" any minor corruption
                cv2.imwrite(img_path, img)
        except Exception as e:
            print(f"Error processing {img_file}: {e}")
            # Optionally, remove files that fail to process
            os.remove(img_path)

# Run the check on each dataset split
check_and_fix_images(val_images_dir)
check_and_fix_images(train_images_dir)
check_and_fix_images(test_images_dir)

print("Image check and fix process complete.")

Corrupted image found: 12_05_21-B30.npy. Attempting to delete.
Corrupted image found: 27_04_21-B21.npy. Attempting to delete.
Corrupted image found: 5_07_21-B12.npy. Attempting to delete.
Corrupted image found: 5_05_21-B14.npy. Attempting to delete.
Corrupted image found: 18_06_21-B5.npy. Attempting to delete.
Corrupted image found: 25_05_21-B33.npy. Attempting to delete.
Corrupted image found: 28_04_21-B11.npy. Attempting to delete.
Corrupted image found: 19_05_21-B26.npy. Attempting to delete.
Corrupted image found: 25_05_21-B16.npy. Attempting to delete.
Corrupted image found: 18_05_21-B45.npy. Attempting to delete.
Corrupted image found: 20_05_21-B1.npy. Attempting to delete.
Corrupted image found: 5_05_21-B8.npy. Attempting to delete.
Corrupted image found: 3_05_21-B7.npy. Attempting to delete.
Corrupted image found: 21_05_21-B23.npy. Attempting to delete.
Corrupted image found: 31_05_21-B5.npy. Attempting to delete.
Corrupted image found: 12_05_21-B9.npy. Attempting to delete.
Co

In [ ]:
model = YOLO('yolov8n.pt')
# Directory containing the training images and labels
train_images_dir = '/content/extracted_images/dataset_split/train/images'
yolo_labels_dir = '/content/extracted_images/yolo/labels'

# List to store validated images
validated_images = []

# Run inference on each image to check for issues
for img_file in os.listdir(train_images_dir):
    img_path = os.path.join(train_images_dir, img_file)
    label_path = os.path.join(yolo_labels_dir, img_file.replace('.jpg', '.txt'))

    # Check if label file exists and is not empty
    if not os.path.exists(label_path) or os.path.getsize(label_path) == 0:
        print(f"Skipping {img_file} due to missing or empty label file.")
        continue

    # Run a quick inference to check for tensor errors
    try:
        img = cv2.imread(img_path)
        results = model(img)  # Run inference
        validated_images.append(img_file)  # Only add if inference succeeds
    except RuntimeError as e:
        if "expected a non-empty list of Tensors" in str(e):
            print(f"Skipping {img_file} due to tensor error during validation.")
        else:
            raise  # Raise any other unexpected errors

print(f"Validated {len(validated_images)} images for training.")



0: 480x640 1 hot dog, 483.2ms
Speed: 18.2ms preprocess, 483.2ms inference, 40.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 knife, 238.6ms
Speed: 5.2ms preprocess, 238.6ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 suitcase, 163.5ms
Speed: 5.6ms preprocess, 163.5ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 175.4ms
Speed: 5.4ms preprocess, 175.4ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 bananas, 1 carrot, 1 dining table, 162.8ms
Speed: 4.9ms preprocess, 162.8ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 163.2ms
Speed: 4.9ms preprocess, 163.2ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 165.2ms
Speed: 5.1ms preprocess, 165.2ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 174.2ms
Speed: 4.8ms pr

In [ ]:
# Step 5: Create YOLOv8 Configuration File
train_images_dir = os.path.abspath(os.path.join(dataset_split_dir, 'train/images'))
val_images_dir = os.path.abspath(os.path.join(dataset_split_dir, 'val/images'))
yolo_config_file = os.path.join(dataset_split_dir, 'yolo_config.yaml')

# Verify that directories exist and contain images
if not os.path.exists(train_images_dir) or len(os.listdir(train_images_dir)) == 0:
    raise FileNotFoundError(f"No images found in training directory: {train_images_dir}")
if not os.path.exists(val_images_dir) or len(os.listdir(val_images_dir)) == 0:
    raise FileNotFoundError(f"No images found in validation directory: {val_images_dir}")

with open(yolo_config_file, 'w', encoding='utf-8') as f:
    f.write(f"""
    train: {train_images_dir}
    val: {val_images_dir}
    nc: {len(categories)}  # Number of classes
    names: {[cat['name'] for cat in categories]}  # Class names
    """)
print(f"YOLOv8 config file created: {yolo_config_file}")


YOLOv8 config file created: /content/extracted_images/dataset_split/yolo_config.yaml


In [ ]:
# Step 6: Train YOLOv8 Model for Fish Detection
  # Load YOLOv8 small model

yolo_config_path_ = '/content/extracted_images/dataset_split'
# Load your config file (replace with your actual path)

# Train with error handling to skip problematic batches
# Run training with error handling
try:
    # Custom training loop
    for epoch in range(8):  # Replace 8 with desired epochs
        print(f"Starting epoch {epoch + 1}...")
        try:
            model.train(data=yolo_config_file, epochs=1, imgsz=640, device='cpu', resume=True)  # Use CPU
        except RuntimeError as e:
            if "expected a non-empty list of Tensors" in str(e):
                print(f"Skipping problematic batch in epoch {epoch + 1}.")
                continue
            elif "NoneType" in str(e):
                print("Encountered a NoneType error, possibly due to an invalid image path.")
                continue
            else:
                raise  # Raise any other unexpected errors
except Exception as main_e:
    print("Error encountered during training:", main_e)

print("Training complete.")

Starting epoch 1...
Ultralytics 8.3.24 🚀 Python-3.10.12 torch-2.5.0+cu121 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/extracted_images/dataset_split/yolo_config.yaml, epochs=500, time=None, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=disk, device=cpu, workers=8, project=YOLOv8, name=yolov8n3, exist_ok=False, pretrained=False, optimizer=SGD, verbose=False, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=yolov8n.pt, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=False, val=True, split=val, save_json=False, save_hybrid=False, conf=0.001, iou=0.7, max_det=300, half=True, dnn=False, plots=True, source=ultralytics/assets/, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_

train: Scanning /content/extracted_images/dataset_split/train/labels... 0 images, 748 backgrounds, 0 corrupt: 100%|██████████| 748/748 [00:00<00:00, 2705.26it/s]

train: WARNING ⚠️ No labels found in /content/extracted_images/dataset_split/train/labels.cache. See https://docs.ultralytics.com/datasets for dataset formatting guidance.


train: New cache created: /content/extracted_images/dataset_split/train/labels.cache
WARNING ⚠️ No labels found in /content/extracted_images/dataset_split/train/labels.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.


train: Caching images (25.0GB Disk): 100%|██████████| 748/748 [05:34<00:00,  2.24it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.20 (you have 1.4.15). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
val: Scanning /content/extracted_images/dataset_split/val/labels... 0 images, 84 backgrounds, 0 corrupt: 100%|██████████| 84/84 [00:00<00:00, 335.27it/s]

val: WARNING ⚠️ No labels found in /content/extracted_images/dataset_split/val/labels.cache. See https://docs.ultralytics.com/datasets for dataset formatting guidance.
val: New cache created: /content/extracted_images/dataset_split/val/labels.cache
WARNING ⚠️ No labels found in /content/extracted_images/dataset_split/val/labels.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.



val: Caching images (2.8GB Disk): 100%|██████████| 84/84 [00:44<00:00,  1.88it/s]


Plotting labels to YOLOv8/yolov8n3/labels.jpg... 
zero-size array to reduction operation maximum which has no identity
optimizer: SGD(lr=0.01, momentum=0.937) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.001), 63 bias(decay=0.0)
Error encountered during training: yolov8n.pt training to 500 epochs is finished, nothing to resume.
Start a new training without resuming, i.e. 'yolo train model=yolov8n.pt'
Training complete.


In [ ]:
# Step 7: Fish Segmentation using Mask R-CNN
mask_rcnn_model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)
mask_rcnn_model.eval()

def mask_rcnn_inference(image_path):
    img = cv2.imread(image_path)
    img_tensor = F.to_tensor(img)
    predictions = mask_rcnn_model([img_tensor])[0]
    for mask in predictions['masks']:
        mask = mask[0].mul(255).byte().cpu().numpy()
        img[mask > 0] = [0, 255, 0]  # Highlight segmented regions in green
    cv2.imshow('Segmented Image', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MaskRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=MaskRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
# Function to calculate fish length
def calculate_fish_length(bbox, img_width, img_height, scale_factor):
    x_min, y_min, box_width, box_height = bbox
    length_in_pixels = max(box_width, box_height)
    length_in_real_units = length_in_pixels * scale_factor
    return length_in_real_units

# Updated inference function with error handling
def infer_fish_length(model, test_image_path, scale_factor):
    try:
        img = cv2.imread(test_image_path)

        # Check if the image is loaded properly
        if img is None:
            print(f"Error: Unable to load image at {test_image_path}. Skipping...")
            return  # Skip this image if not loaded

        results = model(img)

        # Check if results are valid
        if results is None or len(results) == 0:
            print(f"No detection results for {test_image_path}.")
            return  # Skip if there are no results

        for box in results[0].boxes.xywh:  # Assuming YOLOv8 boxes in (x_center, y_center, width, height) format
            x_center, y_center, width, height = box
            length = calculate_fish_length((x_center, y_center, width, height), img.shape[1], img.shape[0], scale_factor)
            print(f"Estimated fish length for {test_image_path}: {length} units")

    except Exception as e:
        print(f"Error during inference for {test_image_path}: {e}")

# Define the paths to the test images
val_images_dir = '/content/extracted_images/dataset_split/val/images'  # Ensure this is correct

# Run inference on test images
scale_factor = 0.01  # Adjust based on your dataset
test_images = [os.path.join(val_images_dir, f) for f in os.listdir(val_images_dir) if f.endswith('.jpg')]

for test_image_path in test_images:
    infer_fish_length(model, test_image_path, scale_factor)

print("Inference complete.")


0: 480x640 (no detections), 182.1ms
Speed: 9.4ms preprocess, 182.1ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 banana, 1 broccoli, 162.7ms
Speed: 5.4ms preprocess, 162.7ms inference, 1.9ms postprocess per image at shape (1, 3, 480, 640)
Estimated fish length for /content/extracted_images/dataset_split/val/images/23_04_21-B7.jpg: 9.002055168151855 units
Estimated fish length for /content/extracted_images/dataset_split/val/images/23_04_21-B7.jpg: 7.362401962280273 units

0: 480x640 1 banana, 1 carrot, 1 hot dog, 1 dining table, 175.9ms
Speed: 5.6ms preprocess, 175.9ms inference, 1.9ms postprocess per image at shape (1, 3, 480, 640)
Estimated fish length for /content/extracted_images/dataset_split/val/images/5_05_21-B50.jpg: 16.383708953857422 units
Estimated fish length for /content/extracted_images/dataset_split/val/images/5_05_21-B50.jpg: 16.300920486450195 units
Estimated fish length for /content/extracted_images/dataset_split/val/images/5_05_21-B

In [ ]:
import gradio as gr

# Load the trained YOLO model
model = YOLO('yolov8n.pt')  # Adjust the path to your model

# Updated inference function
def infer_fish_length(image, scale_factor=0.01):
    img = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)  # Convert from RGB to BGR

    # Run the model on the image
    results = model(img)

    # Check if results are valid
    if results is None or len(results) == 0:
        return "No fish detected."

    lengths = []
    for box in results[0].boxes.xywh:  # Assuming YOLOv8 boxes in (x_center, y_center, width, height) format
        x_center, y_center, width, height = box
        length = calculate_fish_length((x_center, y_center, width, height), img.shape[1], img.shape[0], scale_factor)
        lengths.append(length)

    return lengths

# Create Gradio interface
iface = gr.Interface(
    fn=infer_fish_length,
    inputs=gr.Image(type="pil", label="Upload Fish Image"),
    outputs=gr.Textbox(label="Estimated Fish Lengths (in real units)"),
    title="Fish Length Estimator",
    description="Upload an image of fish, and the model will estimate their lengths."
)

# Launch the interface
iface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c64d4acf68d95c4f70.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
